![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_BERT.ipynb)

# Import ONNX BERT models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for BERT from HuggingFace and they have to be in `Fill Mask` category. Meaning, you cannot use BERT models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.29.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

!pip install -q --upgrade transformers[onnx]==4.29.1 optimum tensorflow

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [bert-base-cased](https://huggingface.co/bert-base-cased) model from HuggingFace as an example and load it as a `ORTModelForFeatureExtraction`, representing an ONNX model.
- In addition to the BERT model, we also need to save the `BertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

from optimum.onnxruntime import ORTModelForFeatureExtraction

MODEL_NAME = "bert-base-cased"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(EXPORT_PATH)

# Create directory for assets and move the tokenizer files.
# A separate folder is needed for Spark NLP.
!mkdir {EXPORT_PATH}/assets
!mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/

Let's have a look inside these two directories and see what we are dealing with:

!ls -l {EXPORT_PATH}

!ls -l {EXPORT_PATH}/assets

## Import and Save BERT in Spark NLP

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [0]:
import sparknlp
# let's start Spark with Spark NLP
# DATABRICKS spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `BertEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `BertEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [0]:
EXPORT_PATH

In [0]:
from sparknlp.annotator import *

# All these params should be identical to the original ONNX model
bert = BertEmbeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["document",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(True)\
    .setDimension(768)\
    .setStorageRef('bert_base_cased')

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [0]:
bert.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [0]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX BERT model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [0]:
! ls -l {MODEL_NAME}_spark_nlp

Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BERT model 😊

In [0]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")

bert_loaded = BertEmbeddings.load(f"{MODEL_NAME}_spark_nlp")\
    .setInputCols(["document",'token'])\
    .setOutputCol("bert")\

pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        bert_loaded
  ])

data = spark.createDataFrame([['William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor,and philanthropist.']]).toDF("text")
model = pipeline.fit(data)
result = model.transform(data)

In [0]:
result.selectExpr("explode(bert.embeddings) as embeddings").show()

That's it! You can now go wild and use hundreds of BERT models from HuggingFace 🤗 in Spark NLP 🚀
